<a href="https://colab.research.google.com/github/MariaShaiina/NLP-2023/blob/main/Lab2_ShainaMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа №2

Лабораторную работу необходимо выполнять в ***Google Colab***. Результат работы (в виде ссылки на notebook) выслать письмом на litvinov.vg@ssau.ru. В теме письма указывать ФИО полностью.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)). Откликами являются значения двух классов positive и negative.

In [1]:
# Импорт модулей и библиотек
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#nltk.download('punkt')
import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.util import ngrams

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import  train_test_split, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import spacy
from sklearn.pipeline import make_pipeline

from google.colab import drive, data_table

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Чтение файла
imdb = pd.read_csv('/content/drive/My Drive/Colab_Notebooks/IMDB Dataset.csv', delimiter = ',')

In [4]:
imdb.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


Данный датасет представляет собой pandas DataFrame с 50 000 записями и двумя столбцами: признак - "review" и откоик - "sentiment"

### Шаг 1. Подготовка данных


#### 1 Data Cleaning

In [5]:
def clean_data(text):
  # Замена всех совпадений шаблона на указанную строку
  cleaner = re.sub(re.compile('<.*?>'), '', text) # удаление тэгов html
  cleaned_review = re.sub('[^A-Za-z0-9]+',' ', cleaner)
  return cleaned_review

In [6]:
imdb_copy = imdb[:5000].copy()
imdb_copy.dropna(inplace=True)
imdb_copy['review'] = imdb_copy['review'].apply(lambda review: clean_data(review))
print(len(imdb_copy))
imdb_copy.head()

5000


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there s a family where a little boy ...,negative
4,Petter Mattei s Love in the Time of Money is a...,positive


In [7]:
X = imdb_copy['review']

#### 2 Токенизация

В качестве исследуемых способов представления необходимо рассмотреть:
#### 1. Bag of words: word counts ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)). Компоненты вектора: частоты или относительные частоты.

#### 2. Bag of words: weird numbers ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). Компоненты вектора: оценки "значимости" слова (например tf-idf).


2.1 CountVectorizer

In [8]:
cnt_vectorizer = CountVectorizer(max_features=15000)
X_Count = cnt_vectorizer.fit_transform(X).toarray()

In [9]:
X_Count.shape

(5000, 15000)

2.2 TfidfVectorizer

В отличие от первого метода Tfidf учитывает также и обратную частоту (idf), чтобы уменьшить вес слов, которые часто встречаются во всей коллекции, так как они могут быть менее информативными для конкретного документа.

In [10]:
tfidf_vectorizer = TfidfVectorizer(max_features=15000)
X_tfidf = tfidf_vectorizer.fit_transform(X).toarray()

In [11]:
X_tfidf.shape

(5000, 15000)

Посмотрим какие слова чаще всего встречаются в нашей матрице

In [ ]:
tfidf_features = tfidf_vectorizer.get_feature_names_out()
print(tfidf_features)

['00' '000' '007' ... 'zp' 'zu' 'zucker']


#### 2.3 Bag of ngrams. Позволяет учитывать положение слов. Компоненты вектора: частоты N-грам. Примитивный подход, но в некоторых задачах он может улучшать качество решения.

In [ ]:
# Создание биграммы
bag_of_bigrams = X.map(lambda text: tuple(nltk.ngrams(text.split(' '), 2)))
bag_of_bigrams[:10]

0    ((One, of), (of, the), (the, other), (other, r...
1    ((A, wonderful), (wonderful, little), (little,...
2    ((I, thought), (thought, this), (this, was), (...
3    ((Basically, there), (there, s), (s, a), (a, f...
4    ((Petter, Mattei), (Mattei, s), (s, Love), (Lo...
5    ((Probably, my), (my, all), (all, time), (time...
6    ((I, sure), (sure, would), (would, like), (lik...
7    ((This, show), (show, was), (was, an), (an, am...
8    ((Encouraged, by), (by, the), (the, positive),...
9    ((If, you), (you, like), (like, original), (or...
Name: review, dtype: object

In [ ]:
# Создание экземпляра TfidfVectorizer с учетом 3-грамм (N=3)
ThTfidfVec = TfidfVectorizer(ngram_range=(3, 3),  max_features=5000)

Th_tfX = ThTfidfVec.fit_transform(X).toarray()

Посмотрим какие пары слов чаще всего встречаются в нашей матрице

In [ ]:
th_features = ThTfidfVec.get_feature_names_out()
print(th_features)

['10 on the' '10 out of' '10 year old' ... 'you would think'
 'your time and' 'your time with']


### Шаг 2. Исследование моделей

В зависимости от способа представления оценить качество классификации как долю правильных ответов на выборке (accuracy). Не забывайте использовать перекрестную проверку ([cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)).

Для каждого из нижеперечисленных моделей необходимо определить оптимальные гиперпараметры ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html))

В датасете IMDB только два отклика: позитивыынй (positive) и негативный (negative). Соответсвенно, мы будем делать бинарную классификацию

In [12]:
# Заменим категориальные данные об откликах на числа
imdb_copy['sentiment']=imdb_copy['sentiment'].replace(['negative','positive'], [0, 1])
imdb_copy.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there s a family where a little boy ...,0
4,Petter Mattei s Love in the Time of Money is a...,1


In [13]:
y = imdb_copy['sentiment']

In [ ]:
print(X.shape, y.shape)

(5000,) (5000,)


**CountVectorizer**

In [14]:
cnX_train, cnX_test, y_train, y_test = train_test_split(X_Count, y, test_size=0.3)

In [15]:
print(cnX_train.shape, y_train.shape)

(3500, 15000) (3500,)


**TF-iDF**

In [16]:
tf_X_train, tf_X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3)

In [17]:
print(tf_X_train.shape, y_train.shape)

(3500, 15000) (3500,)



#### Качество классификации оцениваем для следующих моделей:


#### 1. Метод опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

CountVectorizer

In [ ]:
%%time
# Создадим экземпляр метода опорных векторов
svc_model = SVC(random_state=42)

# Определим гиперпараметры, где С - расстояние между разделяющей гиперплоскостью и ближайшим обуч. примером
parameters = {
              'kernel':('linear', 'rbf'),
              'C':[1, 5, 10]
              }

CV = KFold(n_splits=5, shuffle=True, random_state=63)

grid_searchSVC_CN = GridSearchCV(svc_model, parameters, cv=CV)
grid_searchSVC_CN.fit(cnX_train, y_train)

CPU times: user 1h 35min 17s, sys: 22min 3s, total: 1h 57min 20s
Wall time: 1h 7min 55s


GridSearchCV(cv=KFold(n_splits=5, random_state=63, shuffle=True),
             estimator=SVC(random_state=42),
             param_grid={'C': [1, 5, 10], 'kernel': ('linear', 'rbf')})

TF-iDF

In [ ]:
%%time

CV = KFold(n_splits=5, shuffle=True, random_state=63)

grid_searchSVC_TF = GridSearchCV(svc_model, parameters, cv=CV)
grid_searchSVC_TF.fit(tf_X_train, y_train)

#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

CountVectorizer

In [18]:
# Модель случайного леса
rf_model = RandomForestClassifier(random_state=42)

In [19]:
%%time
# Определение параметров для поиска
rf_parameters = {'n_estimators': [50, 100, 200],
                 'max_depth': [None, 10, 20],
                 'min_samples_split': [2, 5, 10]
                 }

CV = KFold(n_splits=5, shuffle=True, random_state=42)

# Создание объекта GridSearchCV
grid_searchRF_CN = GridSearchCV(rf_model, rf_parameters, cv=CV)

# Обучение GridSearchCV
grid_searchRF_CN.fit(cnX_train, y_train)

CPU times: user 18min 17s, sys: 19.6 s, total: 18min 37s
Wall time: 19min 9s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

TF-iDF

In [20]:
%%time

grid_searchRF_TF = GridSearchCV(rf_model, rf_parameters, cv=CV)
grid_searchRF_TF.fit(tf_X_train, y_train)

CPU times: user 14min 20s, sys: 17.8 s, total: 14min 38s
Wall time: 14min 41s


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]})

### Шаг 3. Оценка качества работы классификаторов и cравнение результатов

Сравнить точность обученных моделей. Найти наиболее точную.

Так как GridSearchCV уже использует объект перекрестной кросс-валидации, то для оценки качества работы классификаторов достаточно просто посчитать оценку на каждой модели

In [26]:
AVAILABLE_NAMES = {"SVC CountVectorizer": cnX_test,
                   "SVC TF-iDF": tf_X_test,
                   "RandomForestClassifier CountVectorizer": cnX_test,
                   "RandomForestClassifier TF-iDF": tf_X_test}

ALL_MODELS = [grid_searchSVC_CN, grid_searchSVC_TF, grid_searchRF_CN, grid_searchRF_TF]

In [27]:
def evaluator(estimator, x_test_data, y_test_data):
    predict = estimator.predict(x_test_data)
    print("Classification report: \n", classification_report(y_test_data, predict, target_names=['Negative','Positive']))
    print("Confusion Matrix: \n", confusion_matrix(y_test_data, predict))
    print("Accuracy: \n", accuracy_score(y_test_data, predict))

In [30]:
# print score
for (model_name, X_test), estimator in zip(AVAILABLE_NAMES.items(), ALL_MODELS):
  print('\n', model_name, '\n')
  evaluator(estimator, X_test, y_test)


 RandomForestClassifier CountVectorizer 

Classification report: 
               precision    recall  f1-score   support

    Negative       0.51      0.64      0.57       751
    Positive       0.52      0.40      0.45       749

    accuracy                           0.52      1500
   macro avg       0.52      0.52      0.51      1500
weighted avg       0.52      0.52      0.51      1500

Confusion Matrix: 
 [[477 274]
 [453 296]]
Accuracy: 
 0.5153333333333333

 RandomForestClassifier TF-iDF 

Classification report: 
               precision    recall  f1-score   support

    Negative       0.82      0.86      0.84       751
    Positive       0.85      0.81      0.83       749

    accuracy                           0.84      1500
   macro avg       0.84      0.84      0.84      1500
weighted avg       0.84      0.84      0.84      1500

Confusion Matrix: 
 [[644 107]
 [139 610]]
Accuracy: 
 0.836


**Вывод**

Таким образом, оба метода работают приерно с одинаковой точностью, при этом классификатор случайного леса показал лучшие результаты по сравнению с методом опорных векторов